# GraphRAG MCP Server Testing

This notebook tests the GraphRAG MCP tools directly and via the HTTP server.

**Prerequisites:**
1. Knowledge graph indexed: `poetry run python -m core.index`
2. Azure OpenAI configured in `.env`

**Optional (for HTTP tests):**

3. MCP server running: `poetry run python run_mcp_server.py`

In [1]:
import sys
import os
from pathlib import Path
from dotenv import load_dotenv

# Add project root to path
sys.path.insert(0, str(Path.cwd().parent))

# Load environment variables
load_dotenv()

# Verify configuration
print("Configuration Check:")
print(f"   Azure OpenAI Endpoint: {os.getenv('AZURE_OPENAI_ENDPOINT')}")
print(f"   Chat Deployment: {os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT', 'gpt-4o')}")
print(f"   Embedding Deployment: {os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT', 'text-embedding-3-small')}")

Configuration Check:
   Azure OpenAI Endpoint: https://maf-graphrag-openai-5woawz.openai.azure.com/
   Chat Deployment: gpt-4o
   Embedding Deployment: text-embedding-3-small


In [2]:
import logging

# ---------------------------------------------------------------------------
# Suppress noisy INFO logs from LiteLLM and GraphRAG internals.
#
# LiteLLM (used internally by GraphRAG) logs every Azure OpenAI API call at
# INFO level. Global search triggers 20+ parallel LLM calls, producing 40+
# log lines per query. GraphRAG also attaches a RichHandler that emits styled
# HTML fragments captured by Jupyter as text/html outputs, inflating the
# .ipynb file size without adding user value.
#
# We keep WARNING level so that actionable messages (e.g. token-limit
# warnings from mixed_context.py) are still visible for traceability.
# ---------------------------------------------------------------------------

logging.basicConfig(level=logging.WARNING)

# LiteLLM — main noise source (INFO on every completion call)
logging.getLogger("litellm").setLevel(logging.WARNING)

# GraphRAG internals — keeps token-limit WARNINGs, removes routine INFO
logging.getLogger("graphrag").setLevel(logging.WARNING)

# httpx / httpcore — suppress HTTP-level request chatter
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)

# OpenAI SDK — suppress retry/connection INFO
logging.getLogger("openai").setLevel(logging.WARNING)

print("\u2705 Logging configured \u2014 noisy INFO logs suppressed (WARNING+ preserved)")

✅ Logging configured — noisy INFO logs suppressed (WARNING+ preserved)


## 1. Test MCP Server Connection (optional)

Only needed if you started the MCP server with `poetry run python run_mcp_server.py`.

In [3]:
import httpx

# Test MCP server health
async def test_mcp_server():
    try:
        async with httpx.AsyncClient() as client:
            response = await client.get("http://localhost:8011/health", timeout=5.0)
            print(f"✅ MCP Server Status: {response.status_code}")
            print(f"   Response: {response.text}")
    except httpx.ConnectError:
        print("❌ MCP Server not running!")
        print("   Start it with: poetry run python run_mcp_server.py")
    except Exception as e:
        print(f"❌ Error: {e}")

await test_mcp_server()

❌ MCP Server not running!
   Start it with: poetry run python run_mcp_server.py


## 2. Test Entity Query Tool

List and search entities directly (no server needed).

In [4]:
from mcp_server.tools.entity_query import entity_query_tool

# List entities by type
result = await entity_query_tool(entity_type="person", limit=5)

if "error" in result:
    print(f"Error: {result['error']}")
else:
    print(f"Found {result['total_found']} person entities (showing {result['returned']}):\n")
    for entity in result["entities"]:
        print(f"  - {entity['name']} ({entity['type']})")

# Search by name
print(f"\nSearching for 'Emily Harrison':")
result = await entity_query_tool(entity_name="Emily Harrison")
if "error" not in result:
    for entity in result["entities"]:
        print(f"  - {entity['name']}: {entity['description'][:100]}...")

Found 31 person entities (showing 5):

  - DR. EMILY HARRISON (PERSON)
  - DAVID KUMAR (PERSON)
  - AMANDA FOSTER (PERSON)
  - DR. JAMES MITCHELL (PERSON)
  - SOPHIA LEE (PERSON)

Searching for 'Emily Harrison':
  - DR. EMILY HARRISON: Dr. Emily Harrison is a distinguished figure in the realm of artificial intelligence and machine lea...
  - EMILY HARRISON: Dr. Emily Harrison serves as the Vice President of AI Research at TechVenture Inc., where she plays ...


## 3. Test Local Search (Entity-Focused)

In [5]:
from mcp_server.tools.local_search import local_search_tool

queries = [
    "Who leads Project Alpha?",
    "Who resolved the GraphRAG index corruption incident?",
    "What technologies are used in Project Beta?",
]

for query in queries:
    print(f"\n{'='*70}")
    print(f"Query: {query}")
    print(f"{'='*70}\n")

    result = await local_search_tool(query)
    if "error" in result:
        print(f"Error: {result['error']}")
    else:
        print(f"Answer: {result['answer'][:500]}...")
        print(f"\nContext: {result['context']}")


Query: Who leads Project Alpha?



Answer: # Leadership of Project Alpha

Project Alpha, a flagship initiative of TechVenture Inc., is guided by a strong leadership team to ensure its successful development and deployment. The leadership is structured across various critical roles within TechVenture.

## Project Lead

**Dr. Emily Harrison**, Head of the AI Research Department, serves as the Project Lead for Project Alpha. Dr. Harrison is responsible for overseeing the overall strategy, research direction, and coordination across the cont...

Context: {'entities_used': 19, 'relationships_used': 0, 'reports_used': 2, 'documents': ['project_alpha.md', 'team_members.md', 'company_org.md', 'customers_partners.md', 'technical_architecture.md']}

Query: Who resolved the GraphRAG index corruption incident?

Answer: ## Resolution of the GraphRAG Index Corruption Incident

The resolution of the GraphRAG Index Corruption incident was primarily led by David Kumar, who served as the incident commander. He played a central role in ma

Answer: ### Technologies Used in Project Beta

Project Beta, an initiative by TechVenture Inc., is an AI-powered healthcare analytics platform aimed at providing advanced data processing and analytics solutions for hospital networks and healthcare providers. The project employs a variety of technologies to ensure compliance, efficiency, and innovation.

#### **Data Processing and Integration**

- **Azure Data Factory** and **HL7 FHIR APIs**: These tools are used to establish a robust data ingestion laye...

Context: {'entities_used': 18, 'relationships_used': 47, 'reports_used': 2, 'documents': ['project_beta.md', 'technology_stack.md', 'customers_partners.md', 'engineering_processes.md', 'project_alpha.md', 'team_members.md']}


## 4. Test Global Search (Thematic)

**Note:** Global search uses map-reduce across all communities, making many parallel LLM calls. May be slow or hit rate limits.

In [6]:
from mcp_server.tools.global_search import global_search_tool

query = "What are the main projects at TechVenture Inc?"
print(f"Query: {query}\n")

result = await global_search_tool(query)
if "error" in result:
    print(f"Error: {result['error']}")
    if "details" in result:
        print(f"Details: {result['details']}")
else:
    print(f"Answer:\n{result['answer']}")
    print(f"\nContext: {result['context']}")

Query: What are the main projects at TechVenture Inc?

Answer:
### Main Projects at TechVenture Inc.

TechVenture Inc. is primarily focused on two flagship projects: **Project Alpha** and **Project Beta**, both of which are crucial in advancing the company's contributions to AI solutions and healthcare analytics [Data: Reports (0, 9, 19, +more)].

#### Project Alpha

Project Alpha is a noteworthy venture at TechVenture, driven by its mission to revolutionize AI technology by integrating multi-modal interactions. This project aims to enhance AI deployment across various industrial applications, which could bring about significant technological advancements [Data: Reports (9, 19)].

Key individuals involved in Project Alpha's success include David Kumar and Jennifer Park. They play crucial roles in overseeing the architectural design and managing the project's infrastructure. Their efforts ensure that Project Alpha remains robust in design and scalable, which are pivotal traits for the p

## 5. Cross-Document Reasoning

In [7]:
complex_queries = [
    "Who are the customers of Project Beta and what incidents affected them?",
    "What is the connection between David Kumar, Sophia Lee, and the GraphRAG incident?",
]

for query in complex_queries:
    print(f"\n{'='*70}")
    print(f"Query: {query}")
    print(f"{'='*70}\n")

    result = await local_search_tool(query)
    if "error" in result:
        print(f"Error: {result['error']}")
    else:
        print(f"Answer: {result['answer'][:500]}...")


Query: Who are the customers of Project Beta and what incidents affected them?



Answer: ## Project Beta Customers

Project Beta, TechVenture Inc.'s AI-powered healthcare analytics platform, serves numerous hospital networks and healthcare providers. It is currently in Phase 4 and has expanded its presence significantly. Key customers include:

1. **Meridian Healthcare System** - A flagship customer with an eight-hospital network utilizing Project Beta [Data: Entities (30); Relationships (35)]. 
2. **Northeast Regional Medical** - A network of five hospitals that have been using Pro...

Query: What is the connection between David Kumar, Sophia Lee, and the GraphRAG incident?



Answer: ## Connection Between David Kumar, Sophia Lee, and the GraphRAG Incident

### David Kumar's Role

David Kumar was the Incident Commander during the GraphRAG Index Corruption incident that occurred on March 18, 2024 [Data: Entities (2)]. In his role, he was responsible for overseeing the incident resolution process and coordinating the actions needed to address the issue. During the incident, David Kumar managed the team efforts to identify and rectify the cause of the corrupt index that led to "...


## Summary

This notebook demonstrates:

1. MCP Server connectivity check
2. Entity query tool (list/search entities)
3. Local search (entity-focused queries)
4. Global search (thematic/community queries)
5. Cross-document reasoning

**Testing via HTTP:** Use [MCP Inspector](https://modelcontextprotocol.io/docs/tools/inspector) for interactive server testing:
```bash
npx @modelcontextprotocol/inspector
```

**Next Steps:**
- Part 3: Supervisor Agent Pattern (agent-framework integration)